In [1]:
import pandas as pd
import csv
import numpy as np
import operator
from sklearn.neighbors import KNeighborsClassifier
from flask import Flask,render_template,request

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [3]:
#data of only islamabad students
isldata = pd.read_csv("islData.csv")
#assigning unique serial nos to students
#isldata['Sr. No'] = isldata.groupby(['Sr. No']).ngroup()

#converting all columns to string
isldata['Warning'] = isldata['Warning'].apply(str)
isldata['CGPA'] = isldata['CGPA'].apply(str)
isldata['SGPA'] = isldata['SGPA'].apply(str)
isldata['Grade Point'] = isldata['Grade Point'].apply(str)
isldata['Credit Hours'] = isldata['Credit Hours'].apply(str)

#course offering csv
course_offr_df = pd.read_csv("course_offerings.csv")

In [4]:
#grouping all columns of same student into 1 row
l=isldata.groupby('Sr. No').agg(' , '.join).reset_index()

In [24]:
#Converting semesters to numbers
l = semesters_to_int(l)

#Creating course grade array
course_grade = all_stu_course_grade(l)

#Creating course_type array
course_type = all_stu_course_type(l)



M:\Python\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [25]:
for i in range(len(course_offr_df)):
    course_offr_df['Course Title'].iloc[i]= course_dict[course_offr_df['Course Title'].iloc[i]][0]


crsofferlist = course_offr_df['Course Title'].tolist()
crsoffer = []
for q in range(len(crsofferlist)):
    for k in course_dict.keys():
        if course_dict[k][0] == crsofferlist[q]:
            crsoffer.append(k)
        
degreePlanList = []
for i in range(len(crsoffer)):
    degreePlanList.append(crsoffer[i])


# EXAMPLE STUDENT

In [26]:
ex_student = isldata[isldata['Sr. No'] == 425]

In [27]:
#example students course grade array
ex_course_grade = ex_stu_course_grade()

#Example students course type array
ex_student, stud = ex_stu_course_type(ex_student)

#finding averages
averages = ex_course_avg()

M:\Python\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
M:\Python\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
M:\Python\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [28]:
can_take_courses = []
can_take_courses_string = []
can_take_courses_new = []
can_take_courses_new_string = []
can_take_courses_repeat = []
can_take_courses_repeat_string = []
must_take_courses = []
must_take_courses_string = []

can_take_courses, can_take_courses_string, can_take_courses_new, can_take_courses_new_string, can_take_courses_repeat, can_take_courses_repeat_string, must_take_courses, must_take_courses_string = ex_suggest_courses()

In [29]:
total_semesters = ex_student['Semester'].iloc[len(ex_student)-1]
if total_semesters == -1:
    total_semesters = ex_student['Semester'].iloc[len(ex_student)-2]

In [30]:
df2 = divide_l_df()

course_grade1 = all_stu_course_grade(df2)

In [31]:
colss=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15',
       '16','17','18','19','20','21','22','23','24',
       '25','26','27','28','29','30','31','32','33',
       '34','35','36','37','38','39','40','41','42', 
       '43','44','45','46','47','48','49','50','51', 
       '52','53','54','55','56','57','58','59','60', 
       '61','62','63','64','65','66','67','68','69',
       '70','71','72','73','74','75','76','77','78',
       '79','80','81','82','83','84','85','86','87',
       '88','89','90','91','92','Student']

In [32]:
numRows=len(course_grade1)
df_ = pd.DataFrame(index=range(numRows),columns=colss)
df_ = df_.fillna(-1) # with 0s rather than NaNs

In [ ]:
for i in range(numRows):
    for j in range(1,df_.shape[1]-1):
        if (course_grade1[i][j]!=-1):
            df_[str(course_grade1[i][j][0])].iloc[i]=course_grade1[i][j][1]
            df_['Student'].iloc[i]=course_grade1[i][0]
        

In [ ]:
neigh=KNeighborsClassifier(n_neighbors=4)
neigh.fit(df_.iloc[:,:92],df_['Student'])

In [ ]:
for i in range(df_.shape[1]-1):
    df_[str(i)] = df_[str(i)].astype(float)

In [ ]:
gplist = []
for i in range(len(ex_course_grade)):
    if ex_course_grade[i] != -1:
        gplist.append(ex_course_grade[i][1])
    else:
        gplist.append(-1)
        
test = [gplist]

In [ ]:
#print("predicted:",neigh.predict(test))
#print("neighbors",neigh.kneighbors(test))

In [ ]:
#total_neighbors = neigh.kneighbors(test)[1][0]
#print(df_['Student'].iloc[total_neighbors[2]])
#for i in range (len(total_neighbors)):
 #   print(total_neighbors[i])
  #  print("student: ",df_['Student'].iloc[total_neighbors[i]])
   # neighbor_crs = df_.iloc[total_neighbors[i]]
    #for j in range (len(neighbor_crs)):
     #   print (neighbor_crs[j], end = ' ')
    #print()

In [ ]:
#total semesters == example student's semesters
n_stud = {} #list of dataframes of neighboring students
for i in range(len(total_neighbors)):
    neighbor = df_['Student'].iloc[total_neighbors[i]]
    n_stud[i] =  isldata[isldata['Sr. No'] == neighbor]

In [ ]:
x = n_stud[1]['Semester']
x=x.tolist()
#print (x)
for i in range(len(n_stud)):
    stu_termsN = n_stud[i]["Semester"]
    stu_termsN = stu_termsN.tolist()
    temp_termN = stu_termsN[0]
    termN = 1
    new_termN = []
    for j in range(len(stu_termsN)):
        if "Summer" in stu_termsN[j]:
            new_termN.append(-1)
        elif temp_termN == stu_termsN[j]:
            new_termN.append(termN)     
        else:
            termN = termN + 1
            temp_termN = stu_termsN[j]
            new_termN.append(termN)
    #print (n_stud[i]["Sr. No"])
    #print (new_termN)
    n_stud[i]["Semester"]=new_termN

In [ ]:
#selected courses of 4 students for student from previous data
selected_courses = {}
for i in range (len(n_stud)):
    selected_courses[i] =n_stud[i].loc[n_stud[i]['Semester']==total_semesters+1]
    

In [ ]:
knn_stu_crse = knn_pred_courses()
rangee = cgpa_range(knn_stu_crse)
mingpa = gpa_calculate(0)
maxgpa = gpa_calculate(1)
musheer_given_courses = musheer_suggestion(knn_stu_crse)

In [ ]:
cgpa_musheer = str(mingpa) + "  " + str(maxgpa)


In [ ]:
x = n_stud[0]['Sr. No'].unique()
matching_student = df_.iloc[:,:92].loc[df_['Student']==x[0]]

count = 0
for i in range(92):
    if(float(matching_student[str(i)])!=-1.0 and float(gplist[i])!=-1.0):
        count = count + 1        

        
arr = ex_student['Course Title'].unique()

a=n_stud[0].loc[n_stud[0]['Semester']>total_semesters]

degree_plan_crs = degree_plan()


In [ ]:
app=Flask(__name__)
selected_courses = []

In [ ]:
@app.route("/")
@app.route("/home")
def home():
    return render_template('home.html', offer=crsoffer, c=cgpa_selected, cantake = can_take_courses_string, musttake = must_take_courses_string)

@app.route("/degreePlan")
def degreePlan():
    return render_template('degreePlan.html', plan=[])

@app.route('/submit', methods=['GET', 'POST'])
def submit():
    if request.method == 'POST':
        checkboxes2 = []
        checkboxes3 = []
        
        for can in can_take_courses_string:
            checkbox = request.form.get('chkbox' + can)
            checkboxes2.append(checkbox)
        print(checkboxes2)
        
        for must in must_take_courses_string:
            checkbox = request.form.get('chkbox' + must)
            checkboxes3.append(checkbox)
        print(checkboxes3)
        
        newlist = []
        stu_choose_self = []
        for i in range(len(checkboxes2)):
            if checkboxes2[i]=='on':
                stu_choose_self.append(can_take_courses_string[i])
                
        ###Append grades in selected_courses-----------------------------------------------
        
        print("Hello")
        stu_choose_self.append("Discrete Structures")
        stu_choose_self = student_choice_courses(stu_choose_self)            
                    
                    
        stu_cgpa = float(ex_student['CGPA'].iloc[len(ex_student)-1])
        
        exS_crses =[]
        for i in range(len(ex_student)):
            exS_crses.append(ex_student['Course Title'].iloc[i])
        len(exS_crses)
    
        stu_choose_self = student_choice_gpa_range_pred(stu_choose_self)

        rangee1 = cgpa_range(stu_choose_self)
        mingpa1 = gpa_calculate(0)
        maxgpa1 = gpa_calculate(1)
    
        self_given_courses = student_given_courses(stu_choose_self)
        cgpa_selected = str(mingpa1) + "  " + str(maxgpa1)
        #-----------------------------------------------------------------------
        return render_template('home.html', offer=crsoffer, c=cgpa, cantake = can_take_courses_string, musttake = must_take_courses_string, sel_courses=self_given_courses)
    
    
@app.route('/ask_suggestion', methods=['GET', 'POST'])
def ask_suggestion():
    if request.method == 'POST':
        return render_template('home.html', offer=crsoffer, c=cgpa_musheer, cantake = can_take_courses_string, musttake = must_take_courses_string, sel_courses = musheer_given_courses)
        
@app.route('/degree_plan', methods=['GET', 'POST'])
def degree_plan():
    if request.method == 'POST':
        return render_template('degreePlan.html', plan=degree_plan_crs)
    


In [ ]:
if __name__=='__main__':
    app.run(debug=True, use_reloader=False)

In [ ]:
chkbox_values = request.form.getlist('chkbox')

# -----------------------------------------------------------------------------------------------------------

# --------------------Functions start from here----------------------------

# Converting term into numerical terms function: semesters_to_int

In [5]:
#for converting terms(semesters) to numbers
#-1 = summer semester

def semesters_to_int(dframe):
    for i in range (len(dframe)):
        stu_terms = dframe['Semester'].iloc[i].split(' , ')
        temp_term = stu_terms[0]
        term = 1
        new_term = []
        for j in range(len(stu_terms)):
            if "Summer" in stu_terms[j]:
                new_term.append(-1)
            elif temp_term == stu_terms[j]:
                new_term.append(term)     
            else:
                term = term + 1
                temp_term = stu_terms[j]
                new_term.append(term)
        new_termm = ' , '.join([str(elem) for elem in new_term])
        dframe.Semester.iloc[i] = new_termm
    return dframe
    

# course_grade contains courses of all the students in a same manner to make mapping easier.         [course_index, student grade, student cgpa, student warning, how many times he has studied the course]
 

In [6]:
#course_grade is a 2d list. Its [i][0]= serial no of student
#It contains tuples of each students studied course and their relevant grades in that courses. mapped with the dictonary provided above.
#Unstudied courses are represented by -1.
#[course value,grade,cgpa,warning,repeat no.]
def all_stu_course_grade(l):
    course_grade = []
    for k in range (len(l)):
        temp=[]
        for j in range (94):
            temp.append(-1)
        course_grade.append(temp)   
    for i in range(len(l)):
        course_grade[i][0]= l['Sr. No'].iloc[i]
        stu_course_titles=l['Course Title'].iloc[i].split(' , ')
        stu_course_grades=l['Grade Point'].iloc[i].split(' , ')
        stu_course_warning=l['Warning'].iloc[i].split(' , ')
        stu_course_cgpa=l['CGPA'].iloc[i].split(' , ')
        for j in range(0,len(stu_course_grades)):
            index = course_dict[stu_course_titles[j]][0]
            #[index, student grade, student cgpa, student warning, how many times he has studied the course]
            if(course_grade[i][index+1]==-1):
                course_grade[i][index+1]=[index,stu_course_grades[j],stu_course_cgpa[j],stu_course_warning[j],0]    
            else:
                course_grade[i][index+1][1] = stu_course_grades[j]
                course_grade[i][index+1][4]=course_grade[i][index+1][4]+1
                
    return course_grade
            


# course_type contains information about how many specific types of courses a student has studied and in which semester

In [7]:
def all_stu_course_type(l):
    course_type=[]
    for i in range(len(l)):
        stu_course_titles = l['Course Title'].iloc[i].split(' , ')
        stu_termss = l['Semester'].iloc[i].split(' , ')
        cs_courses = []
        cl_courses = []
        mg_courses = []
        mt_courses = []
        ee_courses = []
        el_courses = []
        ss_courses = []
        sl_courses = []
        for j in range(1,len(stu_course_titles)):
            course = course_dict[stu_course_titles[j]][0]
            typee = course_dict[stu_course_titles[j]][2]
            if typee == "CS":
                cs_courses.append([course,typee,stu_termss[j]])

            if typee == "CL":
                cl_courses.append([course,typee,stu_termss[j]])

            if typee == "EE":
                ee_courses.append([course,typee,stu_termss[j]])

            if typee == "EL":
                el_courses.append([course,typee,stu_termss[j]])

            if typee == "SS":
                ss_courses.append([course,typee,stu_termss[j]])

            if typee == "SL":
                sl_courses.append([course,typee,stu_termss[j]])

            if typee == "MG":
                mg_courses.append([course,typee,stu_termss[j]])

            if typee == "MT":
                mt_courses.append([course,typee,stu_termss[j]])

        c=[]        
        c.append(cs_courses)
        c.append(cl_courses)
        c.append(ee_courses)
        c.append(el_courses)
        c.append(mg_courses)
        c.append(mt_courses)
        c.append(ss_courses)
        c.append(sl_courses)
        course_type.append(c)
    return course_type

# Example students course_grade_array

In [8]:
#course_grade is a 2d list. Its [i][0]= serial no of student
#It contains tuples of each students studied course and their relevant grades in that courses. mapped with the dictonary provided above.
#Unstudied courses are represented by -1.
#[course value,grade,cgpa,warning,repeat no.]

def ex_stu_course_grade():
    ex_course_grade = []

    for j in range (92):
        ex_course_grade.append(-1)

    for i in range(len(ex_student)):
        index = course_dict[ex_student['Course Title'].iloc[i]][0]
        #[index, student grade, student cgpa, student warning, how many times he has studied the course]
        if(ex_course_grade[index]==-1):
            ex_course_grade[index]=[index,ex_student['Grade Point'].iloc[i],ex_student['CGPA'].iloc[i],ex_student['Warning'].iloc[i],0]    
        else:
            ex_course_grade[index][1] = ex_student['Grade Point'].iloc[i]
            ex_course_grade[index][4] = ex_course_grade[index][4]+1
    return ex_course_grade


# Converting its term to numeirc values. Creating its course_type array and replacing course title names with dictionary values

In [9]:
def ex_stu_course_type(ex_student):
    #for converting terms(semesters) to numbers
    #-1 = summer semester
    temp_term = ex_student['Semester'].iloc[0]
    term = 1
    for i in range (len(ex_student)):
        if temp_term == ex_student['Semester'].iloc[i]:
            if "Summer" in temp_term:
                ex_student['Semester'].iloc[i] = -1

            else:
                ex_student['Semester'].iloc[i] = term

        else:
            temp_term = ex_student['Semester'].iloc[i]
            if "Summer" in temp_term:
                ex_student['Semester'].iloc[i] = -1

            else:
                term = term + 1
                ex_student['Semester'].iloc[i] = term

    #mapping specific student wrt course types

    cs_courses = []
    cl_courses = []
    mg_courses = []
    mt_courses = []
    ee_courses = []
    el_courses = []
    ss_courses = []
    sl_courses = []
    for j in range(len(ex_student)):
        course = course_dict[ex_student['Course Title'].iloc[j]][0]
        typee = course_dict[ex_student['Course Title'].iloc[j]][2]
        if typee == "CS":
            cs_courses.append([course,typee,float(ex_student['Grade Point'].iloc[j]),ex_student['Semester'].iloc[j]])

        if typee == "CL":
            cl_courses.append([course,typee,float(ex_student['Grade Point'].iloc[j]),ex_student['Semester'].iloc[j]])

        if typee == "EE":
            ee_courses.append([course,typee,float(ex_student['Grade Point'].iloc[j]),ex_student['Semester'].iloc[j]])

        if typee == "EL":
            el_courses.append([course,typee,float(ex_student['Grade Point'].iloc[j]),ex_student['Semester'].iloc[j]])

        if typee == "SS":
            ss_courses.append([course,typee,float(ex_student['Grade Point'].iloc[j]),ex_student['Semester'].iloc[j]])

        if typee == "SL":
            sl_courses.append([course,typee,float(ex_student['Grade Point'].iloc[j]),ex_student['Semester'].iloc[j]])

        if typee == "MG":
            mg_courses.append([course,typee,float(ex_student['Grade Point'].iloc[j]),ex_student['Semester'].iloc[j]])

        if typee == "MT":
            mt_courses.append([course,typee,float(ex_student['Grade Point'].iloc[j]),ex_student['Semester'].iloc[j]])


    stud=[]        
    stud.append(cs_courses)
    stud.append(cl_courses)
    stud.append(ee_courses)
    stud.append(el_courses)
    stud.append(mg_courses)
    stud.append(mt_courses)
    stud.append(ss_courses)
    stud.append(sl_courses)

    #print (cs_courses)

    for i in range(len(ex_student)):
        ex_student['Course Title'].iloc[i]= course_dict[ex_student['Course Title'].iloc[i]][0]
        
    return ex_student, stud

In [10]:
#taking avgs of course types of a student to see which type of course they perfrom better in\
#[CS,CL,EE,EL,MG,MT,SS,SL]
def ex_course_avg():
    averages=[-1,-1,-1,-1,-1,-1,-1,-1]
    for i in range (len(stud)):
        temp_avg = 0
        if(len(stud[i])>0):
            for j in range (len(stud[i])):
                temp_avg = temp_avg + stud[i][j][2]
            averages[i]=temp_avg/len(stud[i])

    return averages

In [11]:
#Coverting all students' semesters equal to the incoming student's semesters

def divide_l_df():
    same_semesters_new = []
    df2 = pd.DataFrame()
    for i in range (len(l)):
        #print (i)
        stu_terms = l['Semester'].iloc[i].split(' , ')  #student's semesters
        stu_cc = l['Course Code'].iloc[i].split(' , ')  #student's course code
        stu_ct = l['Course Title'].iloc[i].split(' , ')  #student's course title
        stu_ch = l['Credit Hours'].iloc[i].split(' , ')  #student's credit hours
        stu_g = l['Grade'].iloc[i].split(' , ')  #student's grade
        stu_gp = l['Grade Point'].iloc[i].split(' , ')  #student's grade point
        stu_sgpa = l['SGPA'].iloc[i].split(' , ')  #student's sgpa
        stu_cgpa = l['CGPA'].iloc[i].split(' , ')  #student's cgpa
        stu_warning = l['Warning'].iloc[i].split(' , ')  #student's warning

        if int(stu_terms[len(stu_terms)-1])> total_semesters+1:

            same_semesters = []
            same_CourseCode = []
            same_CourseTitle = []
            same_CreditHours = []
            same_Grade = []
            same_GradePoint = [] 
            same_SGPA = []
            same_CGPA = []
            same_Warning = []
            for j in range(len(stu_terms)):
                if int(stu_terms[j])<total_semesters+1:
                    same_semesters.append(stu_terms[j])
                    same_CourseCode.append(stu_cc[j])
                    same_CourseTitle.append(stu_ct[j])
                    same_CreditHours.append(stu_ch[j])
                    same_Grade.append(stu_g[j])
                    same_GradePoint.append(stu_gp[j])
                    same_SGPA.append(stu_sgpa[j])
                    same_CGPA.append(stu_cgpa[j])
                    same_Warning.append(stu_warning[j])


            #same_semesters_new.append(same_semesters)
            same_semesters = ' , '.join([str(elem) for elem in same_semesters])
            same_CourseCode = ' , '.join([str(elem) for elem in same_CourseCode])
            same_CourseTitle = ' , '.join([str(elem) for elem in same_CourseTitle])
            same_CreditHours = ' , '.join([str(elem) for elem in same_CreditHours])
            same_Grade = ' , '.join([str(elem) for elem in same_Grade])
            same_GradePoint = ' , '.join([str(elem) for elem in same_GradePoint])
            same_SGPA = ' , '.join([str(elem) for elem in same_SGPA])
            same_CGPA = ' , '.join([str(elem) for elem in same_CGPA])
            same_Warning = ' , '.join([str(elem) for elem in same_Warning])

            df2= df2.append({"Sr. No":l['Sr. No'].iloc[i], 
                            "Semester":same_semesters,
                               "Course Code":same_CourseCode,
                               "Course Title":same_CourseTitle,
                               "Credit Hours":same_CreditHours,
                               "Grade":same_Grade,
                               "Grade Point":same_GradePoint,
                               "SGPA":same_SGPA,
                               "CGPA":same_CGPA,
                               "Warning":same_Warning},ignore_index=True)
        
    return df2

# In must_take all F courses that are present in course offering are added

In can_take all courses of course offering except for previously studied pre reqs are added

In can_take_repeat courses that are present in the student transcrpt as well as the curse offering are added

In can_take_new all new courses from course offering are added

In [12]:
def ex_suggest_courses():
    #in case of summer semester
    semester_name = 'Summer 2019'
    stu_offer_title = ex_student['Course Title'].tolist()
    offer_title = course_offr_df['Course Title'].tolist()
    can_take_courses = []
    #for taking all courses
    can_take_courses_repeat = []
    can_take_courses_new = []
    #for handling only F and W cases 
    must_take_courses = []
    prereq_remove = []

    #no_of_courses = input("How many courses do you want to take in Summer? ")

    for i in range(len(ex_course_grade)):
        if (ex_course_grade[i] != -1):
            if (ex_course_grade[i][1]=='0.0'):
                for d in range(len(course_lab)):
                    #applying the advisory rule that if student fails course then its lab is must take
                    if ex_course_grade[i][0] == course_lab[d][0]:
                        must_take_courses.append(course_lab[d][1])
                must_take_courses.append(ex_course_grade[i][0])
            #can_take_courses.append(ex_course_grade[i][0])

    remove_courses = []
    for i in range (len(must_take_courses)):
        if must_take_courses[i] not in offer_title:
            remove_courses.append(must_take_courses[i])
            break

    for i in range(len(must_take_courses)):
         for k in course_dict.keys():
            if course_dict[k][0] == must_take_courses[i]:
                if course_dict[k][1] == 0:
                    remove_courses.append(must_take_courses[i])

    remove_courses = list(set(remove_courses))

    for i in range(len(remove_courses)):
        must_take_courses.remove(remove_courses[i])

    must_take_courses = list(dict.fromkeys(must_take_courses))    

    #------------------------------------------------------------

    #Filling can_take with courses from course offering except for pre req
    can_take_courses = offer_title

    #find studied preqs in transcript
    for i in range(len(can_take_courses)):
        for j in range(len(pre_req)):
            if pre_req[j][1] == can_take_courses[i]:
                prereq_remove.append(pre_req[j][0])


    #removing prereqs from can_take_course
    prereq_remove = list(dict.fromkeys(prereq_remove))    
    for i in range(len(prereq_remove)):
        if prereq_remove[i] in can_take_courses_repeat:
            can_take_courses.remove(prereq_remove[i])


    #repeat courses only
    for i in range(len(can_take_courses)):
        if can_take_courses[i] in stu_offer_title:
            can_take_courses_repeat.append(can_take_courses[i])
        else:
            can_take_courses_new.append(can_take_courses[i])


    #looking at the tendency of student

    #--------------------------------------------------------


    can_take_courses_string = []
    for q in range(len(can_take_courses)):
        for k in course_dict.keys():
            if course_dict[k][0] == can_take_courses[q]:
                can_take_courses_string.append(k)
                break


    can_take_courses_repeat_string = []
    for q in range(len(can_take_courses_repeat)):
        for k in course_dict.keys():
            if course_dict[k][0] == can_take_courses_repeat[q]:
                can_take_courses_repeat_string.append(k)
                break


    can_take_courses_new_string = []
    for q in range(len(can_take_courses_new)):
        for k in course_dict.keys():
            if course_dict[k][0] == can_take_courses_new[q]:
                can_take_courses_new_string.append(k)
                break

    must_take_courses_string = []
    for q in range(len(must_take_courses)):
        for k in course_dict.keys():
            if course_dict[k][0] == must_take_courses[q]:
                must_take_courses_string.append(k)
                break

    #print("Can take courses: ",can_take_courses)
    #print("Must take courses: ",must_take_courses)

    #print("Can take courses: ")
    #print(*can_take_courses_string,sep='\n')
    #print('---------------------------------------------------------------------------')
    #print("Can take courses repeat: ")
    #print(*can_take_courses_repeat_string,sep='\n')
    #print('---------------------------------------------------------------------------')

    #print("Can take courses new: ")
    #print(*can_take_courses_new_string,sep='\n')
    #print('---------------------------------------------------------------------------')

    #print("\nMust take courses: ")
    #print(*must_take_courses_string,sep='\n')
    
    return can_take_courses, can_take_courses_string, can_take_courses_new, can_take_courses_new_string, can_take_courses_repeat, can_take_courses_repeat_string, must_take_courses, must_take_courses_string


In [13]:
#converting to numeric values of Course Title
def knn_pred_courses():
    knn_stu_crse = list(selected_courses[0]['Course Title'])
    knn_stu_grade = list(selected_courses[0]['Grade Point'])
    knn_stu_crdthr = list(selected_courses[0]['Credit Hours'])

    for i in range (len(knn_stu_crse)):
        if course_dict[knn_stu_crse[i]][0] == 53:
            knn_stu_crse[i] = [13, float(knn_stu_grade[i]),float(knn_stu_crdthr[i])]
        elif course_dict[knn_stu_crse[i]][0] == 84:
            knn_stu_crse[i] = [63, float(knn_stu_grade[i]),float(knn_stu_crdthr[i])]
        else:
            knn_stu_crse[i] = [course_dict[knn_stu_crse[i]][0], float(knn_stu_grade[i]),float(knn_stu_crdthr[i])]


    return knn_stu_crse




In [14]:
def cgpa_range(array):
    rangee = []
    for i in range(len(array)):
        if array[i][1] == 4.0:
            rangee.append([3.67,4.0])
        else:
            rangee.append([array[i][1] - .33,array[i][1] + .33])

    return rangee


In [15]:
def gpa_calculate(type_gpa):
    ttl_crdit = 0
    sumgp = 0
    for i in range(len(rangee)):
        sumgp = sumgp + knn_stu_crse[i][2]* rangee[i][type_gpa]
        ttl_crdit = ttl_crdit + knn_stu_crse[i][2]

    gpa = sumgp/ttl_crdit
    return gpa


In [16]:
def musheer_suggestion(knn_stu_crse):
    musheer_given_courses = []
    for j in range(len(knn_stu_crse)):
        for i in course_dict.keys():
            if course_dict[i][0] == knn_stu_crse[j][0]:
                if i == "Technical and Business Writing":
                    musheer_given_courses.append( "Discrete Structures" +"   "+str(rangee[j][0]) + " - " + str(rangee[j][1]))
                else:
                    musheer_given_courses.append( i +"   "+str(rangee[j][0]) + " - " + str(rangee[j][1]))

    return musheer_given_courses


In [17]:
def student_choice_courses(stu_choose_self):
    for i in range (len(stu_choose_self)):
        stu_choose_self[i] = course_dict[stu_choose_self[i]][0] 
        for i in range(len(course_offr_df)):
            for j in range(len(stu_choose_self)):
                if int(course_offr_df['Course Title'].iloc[i]) == stu_choose_self[j]:
                    stu_choose_self[j] = [stu_choose_self[j],course_offr_df['Credit Hours'].iloc[i]]
    return stu_choose_self
        

In [18]:
def student_choice_gpa_range_pred(stu_choose_self):    
    credits_earned = 0
    to_earn_credits = 0
    u_stu_offer_title = list(dict.fromkeys(stu_offer_title))
    for i in range(len(u_stu_offer_title)):
        for j in range(len(ex_student)):
            if u_stu_offer_title[i] == ex_student['Course Title'].iloc[j]:
                credits_earned = credits_earned + float(ex_student['Credit Hours'].iloc[j])
                break
                    
    for i in range(len(stu_choose_self)):
        for j in range(len(ex_student)):
            if stu_choose_self[i][0] == ex_student['Course Title'].iloc[j]:
                to_earn_credits = to_earn_credits + float(ex_student['Credit Hours'].iloc[j])
                print(to_earn_credits)
                break



                    
    for i in range(len(stu_choose_self)):        
        to_earn_credits = to_earn_credits + stu_choose_self[i][1]
        print(to_earn_credits)



    prev_thing = 0.0
    credit_earnedd = 0

    for i in range(len(ex_student)):
        if float(ex_student['Course Title'].iloc[i])  not in stu_choose_self:
            prev_thing = prev_thing + (float(ex_student['Credit Hours'].iloc[i])*float(ex_student['Grade Point'].iloc[i]))
            credit_earnedd = credit_earnedd + float(ex_student['Credit Hours'].iloc[i])


    x =(stu_cgpa*(credits_earned+to_earn_credits) - prev_thing)/to_earn_credits

    strx= str(x)
    if int(strx[2]) - 3 < 6 - int(strx[2]):
        new = strx[0]+'.33'

    if int(strx[2]) - 3 > 6 - int(strx[2]):
        new = strx[0]+'.67'


    for i in range(len(stu_choose_self)):
        stu_choose_self[i] = [stu_choose_self[i][0],float(new),stu_choose_self[i][1]]
    
    return stu_choose_self


In [19]:
def student_given_courses(stu_choose_self):
    self_given_courses = []
    for j in range(len(stu_choose_self)):
        for i in course_dict.keys():
            if course_dict[i][0] == stu_choose_self[j][0]:
                self_given_courses.append( i +"   "+str(rangee1[j][0]) + " - " + str(rangee1[j][1]))
    return self_given_courses


In [20]:
def degree_plan():
    #total_semesters
    degree_plan_crs = []

    #print (len(n_stud[0]))
    for i in range(len(a)):

        crs_str =a['Course Title'].iloc[i]
        #print (crs_str)
        #crs_str.append(str(a['Semester'].iloc[i]) +"  "+ crs_str)

        course_val = course_dict[crs_str][0]
        for k in course_dict.keys():
            if course_dict[k][0] == course_val:
                if course_dict[k][1] == 0:
                    if course_dict[k][2] == "MG":
                        crs_str ="MG Elective"
                        degree_plan_crs.append(str(a['Semester'].iloc[i]) +"  "+ crs_str)

                    elif course_dict[k][2] == "CS":
                        crs_str ="CS Elective"
                        degree_plan_crs.append(str(a['Semester'].iloc[i]) +"  "+ crs_str)
                    else:
                        degree_plan_crs.append(str(a['Semester'].iloc[i]) +"  "+ crs_str)
                else:
                    degree_plan_crs.append(str(a['Semester'].iloc[i]) +"  "+ crs_str)


    degree_plan_crs.remove('6  Numerical Methods')
    degree_plan_crs.append('6  Discrete Structures')
    degree_plan_crs
    return degree_plan_crs

# --------------------------Functions end here------------------------------------

# ----------------------------------------------------------------------------------------------------------

# Mapping relationship between courses and their respective labs

In [21]:
course_lab =[
    [2,1], #itc, itc lab
    [6,5], #cp, cp lab
    [8,7], #eng, eng lab I
    [9,10], #dld, dld lab
    [17,16], #coal, coal lab
    [22,23], #os, os lab
    [20,44], #ds, ds lab
    [29,28], #ooad, ooad lab
    [32,33],#cnet, cnet lab
    [35,40]#db, db lab
]

# Dictonary containing all courses a student can study.
It also creates mapping for similar courses that have changed names.
[course_index, core or elective, type of course]

In [22]:
#dictonary of courses where each course is given a unique id
#Also mapping for changed course i.e. ITC and PF
#[Course Code, Core = 1, Elective=0, COurse Type]
course_dict={
"Basic Electronics":[0,1,"EE"],
'Applied Physics':[0,1,"EE"],
'Physics - I':[0,1,"EE"],
"Introduction to Computing Lab":[1,1,"CL"],
'Introduction to Computing - Lab':[1,1,"CL"],
"Programming Fundamentals - Lab":[1,1,"CL"],
"Introduction to Computing":[2,1,"CS"],
'Programming Fundamentals':[2,1,"CS"],
"Islamic and Religious Studies":[3,1,"SS"],
"Calculus and Analytical Geometry":[4,1,"MT"],
'Calculus - I':[4,1,"MT"],
'Object Oriented Programming - Lab':[5,1,"CL"],
"Computer Programming Lab":[5,1,"CL"],
"Computer Programming":[6,1,"CS"],
"Object Oriented Programming":[6,1,"CS"],    
"English Language":[7,1,"SS"],
"English Composition and Comprehension":[7,1,"SS"],
"English Language Lab":[8,1,"SL"],  
'English I - Lab':[8,1,"SL"],
"English Composition and Comprehension - Lab":[8,1,"SL"],
"Digital Logic Design":[9,1,"EE"],
'Computer Logic Design':[9,1,"EE"],
"Digital Logic Design - Lab":[10,1,"EL"],
'Computer Logic Design Lab':[10,1,"EL"],
"Calculus - II":[11,1,"MT"],
'Differential Equations (Cal II)':[11,1,"MT"],
'Intro to Info. and Comm. Technologies':[12,1,"CL"],
'Semantic Web Technologies':[13,0,"CS"],
"Pakistan Studies":[14,1,"SS"],
'History of Pakistan':[14,1,"SS"],
"English Composition":[15,1,"SS"],
'Communication & Presentation Skills':[15,1,"SS"],
"Comp. Organization and Assembly Lang. Lab":[16,1,"EL"],
"Comp. Organization and Assembly Lang-Lab":[16,1,"EL"],
"Comp. Organization and Assembly Lang":[17,1,"EE"],
"Comp. Organization and Assembly Lang.":[17,1,"EE"],
"Business Communication - I":[18,1,"SS"],
"Linear Algebra":[19,1,"MT"],
"Data Structures":[20,1,"CS"],
"Discrete Structures":[21,1,"CS"],
"Operating Systems":[22,1,"CS"],
"Operating Systems Lab":[23,1,"CL"],
"Data Mining":[24,0,"CS"],
"Probability and Statistics":[25,1,"MT"],
"Theory of Automata":[26,1,"CS"],
"Computing Methods - I":[27,1,"MT"],
"Object Oriented Analysis and Design":[28,1,"CS"],
"Object Oriented Analysis and Design Lab":[29,1,"CL"],
"Computer Architecture":[30,1,"EE"],
"Fundamentals of Management":[31,1,"MG"],
"Computer Networks":[32,1,"CS"],
"Computer Networks Lab":[33,1,"CL"],
"Advanced Programming":[34,0,"CS"],
"Database Systems":[35,1,"CS"],
"Software Engineering":[36,1,"CS"],
"Design and Analysis of Algorithms":[37,1,"CS"],
"Web Programming":[38,0,"CS"],
"Entrepreneurship":[39,0,"MG"],
"Database Systems Lab":[40,1,"CL"],
"Numerical Methods":[41,1,"MT"],
'Numerical Analysis':[41,1,"MT"],
'Communication & Presentation Skills - Lab':[42,1,"SL"],
"Mobile Computing":[43,0,"CS"],
'Data Structures - Lab':[44,1,"CL"],
"Technical and Business Writing":[45,1,"SS"],
"Principles of Marketing":[46,0,"MG"],
'Bio-Informatics':[47,0,"CS"],
'Introduction to Bioinformatics':[47,0,"CS"],
'Organizational Behaviour and Leadership':[48,0,"MG"],
'Digital Image Processing':[49,0,"CS"],
'Psychology':[50,0,"SS"],
'Advanced Databases':[51,0,"CS"],
'Network Security':[52,0,"CS"],
'Management Info. Systems':[53,0,"CS"],
'Artificial Intelligence':[54,1,"CS"],
'Software for Mobile Devices':[55,0,"CS"],
'Project - I':[56,1,"CS"],
'Professional Issues in IT':[57,1,"CS"],
'Technical Report Writing':[58,1,"SS"],
'Project - II':[59,1,"CS"],
'Sociology':[60,0,"SS"],
'Marketing Management':[61,0,"MG"],
'Human Computer Interaction':[62,1,"CS"],
'Deep Learning for Perception':[63,0,"CS"],
'Data Warehousing':[64,0,"CS"],
'Cryptography & Data Security':[65,0,"CS"],
'Introduction to Block Chain and Cryptocurrency':[66,0,"CS"],
'Technology Management':[67,0,"MG"],
'Critical Thinking':[68,0,"SS"],
'Text Mining':[69,0,"CS"],
'Software Testing':[70,0,"CS"],
'Intro. to Software Project Management':[71,0,"CS"],
'Physics for Engineers':[72,1,"EE"],
'Conflict Resolution and Negotiation Skills':[73,0,"MG"],
'Multiagent Systems':[74,0,"CS"],
'Arabic Language':[75,0,"SS"],
'Applied Evolutionary Computing':[76,0,"CS"],
'Remote Sensing and GIS':[77,0,"CS"],
'Concurrent and Distributed Systems':[77,0,"CS"],
'Information Retrieval':[78,0,"CS"],
'Compiler Construction':[79,0,"CS"],
'Circuit Analysis - I':[80,0,"EE"],
'Enterprise Information System':[81,0,"CS"],
'Information Processing Techniques':[82,0,"CS"],
'User Experience Engineering':[83,0,"CS"],
'Computer Graphics':[84,0,"CS"],
'Information Systems Security':[85,0,"CS"],
'Fundamentals of Computer Vision':[86,0,"CS"],
'Introduction to the Internet of Things':[87,0,"CS"],
'Finance and Accounting':[88,0,"MG"],
'Accounting and Information System':[89,0,"MG"],    
'Data Science':[90,0,"CS"],
'Introduction to Cloud Computing':[91,0,"CS"],
}

# A 2d array for mapping pre_reqs of all courses

In [23]:
#Dependency graph of pre reqs mapped using course dictonary
pre_req = [[1,5], #ITC lab, CP lab
           [1,6], #itc lab, cp
           [2,6], #itc, cp
           [2,5], #itc, cp lab
           [5,20], #cp lab, ds
           [6,20], #cp, ds  #test for only 2 pre req
           [5,44], #cp lab, ds lab
           [6,44], #cp, ds lab
           [20,35], #ds, db
           [20,22], #ds, os
           [20,32], #ds, cnet
           [20,37], #ds, algo
           [20,28], #ds, ooad
           [20,40], #ds, db lab
           [20,23], #ds, os lab
           [20,29], #ds, ooad lab
           [20,33], #ds, cnet lab
           [44,35], #ds lab, db
           [44,22], #ds lab, os
           [44,32], #ds lab, cnet
           [44,37], #ds lab, algo
           [44,28], #ds lab, ooad
           [44,23], #ds lab, os lab
           [44,40], #ds lab, db lab
           [44,33], #ds lab, cnet lab
           [44,29], #ds lab, ooad lab
           [35,51], #db, adv db
           [40,51], #db lab, adv db
           [35,62], #db, hci
           [40,62], #db lab, hci
           [32,52], #cnet, network security
           [33,52], #cnet lab, network security
           [28,36], #ooad, SE
           [29,36], #ooad lab, SE
           #EE
           [9,17], #dld, coal
           [9,16], #dld, coal lab
           [10,17], #dld lab, coal
           [10,16], #dld lab, coal lab
           [17,30], #coal, CA
           [16,30], #coal lab, CA
           #MT
           [4,11], #cal-I, cal-II
           [19,63], #LA, DL
           [19, 49], #LA, DIP
           [21,26], #Discrete Structures, Automata
           #SS
           [7,15], #English Lang, English composition
           [8,15], #English lang lab, Ennglish comp
           [15,58], #English comp, TRW
           [15,45]] # English Comp, TBW
